In [ ]:
import requests
from bs4 import BeautifulSoup
import json

url = 'https://pueblosoriginarios.com/lenguas/mayas.php'

response = requests.get(url)

if response.status_code == 200:

  soup = BeautifulSoup(response.text,'html.parser')

  table = soup.find('table',{'id':'diccionario'})

  rows = table.find_all('tr')

  diccionario_data = []

  for row in rows[1:]:
    cols = row.find_all('td')

    if len(cols) == 2:
      maya_word = cols[0].text.strip()
      spanish_word = cols[1].text.strip()
      diccionario_data.append({'Maya':maya_word, 'Español':spanish_word})

  json_data = json.dumps(diccionario_data, indent=4, ensure_ascii=False)

  with open('diccionario_maya.json', 'w', encoding='utf-8') as file:
    file.write(json_data)

  print("Datos extraidos y guardados en diccionario_maya.json")

else:

  print("error al conectarse a la web")

Datos extraidos y guardados en diccionario_maya.json


  # PANDAS

In [ ]:
import pandas as pd

# Cargar klos datos desde el archivo json
with open('diccionario_maya.json', 'r', encoding='utf-8') as file:
  diccionario_data = json.load(file)

# Crear un dataframe
df = pd.DataFrame(diccionario_data)

# Crear el excel a partir del DataFrame
df.to_excel('diccionario_maya.xlsx', index=False)

print("Datos guardados en el diccionario_maya.xlsx")


Datos guardados en el diccionario_maya.xlsx


In [ ]:
import pandas as pd
import json

# Cargar klos datos desde el archivo json
with open('diccionario_maya.json', 'r', encoding='utf-8') as file:
  diccionario_data = json.load(file)

# Crear un dataframe
df = pd.DataFrame(diccionario_data)

# 1. Estadistica descriptiva
descripcion_maya = df['Maya'].describe()
descripcion_espanol = df['Español'].describe()

print("Estadística descriptiva de las palabras Maya")
print(descripcion_maya)

print("Estadística descriptiva de las palabras Español")
print(descripcion_espanol)

# 2. Filtrar datos

palabras_con_a = df[df['Maya'].str.startswith('a',na=False)]

print("Palabras que comienzan con 'a' :")
print(palabras_con_a)

# 3. Ordenar datos

df_ordenado = df.sort_values(by='Maya',key=lambda x: x.str.len(), ascending=True)
print("Palabras ordenadas:")
print(df_ordenado)

# 4. Contar duplicados

duplicados_maya = df[df.duplicated(subset='Maya', keep=False)]

print("Palabras que se duplican en Maya")
print(duplicados_maya)

Estadística descriptiva de las palabras Maya
count      4425
unique     3574
top       ookol
freq          9
Name: Maya, dtype: object
Estadística descriptiva de las palabras Español
count      4425
unique     4406
top       sanar
freq          2
Name: Español, dtype: object
Palabras que comienzan con 'a' :
                         Maya        Español
0                           a  tu (posesivo)
1                           a           tuya
2                           a           tuyo
3                       a’ake       apellido
4                        a’al       advertir
..                        ...            ...
143                   ayik’al           rico
144              ayik’al máak        burgués
145  ayik’al, halach wíinikil    aristócrata
146                  ayim kay          catán
147                  ayim kay    pejelagarto

[99 rows x 2 columns]
Palabras ordenadas:
                                                   Maya            Español
0                                

ETL SCRIPT DE JSON A MYSQL

In [ ]:
!pip install mysql.connector

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 77.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for mysql.connector: filename=mysql_connector-2.2.9-cp310-cp310-linux_x86_64.whl size=247952 sha256=1126b1c4a813e168f74a6eb0d13d63f46e936806a9bfaf340eb4e6b638b978f5
  Stored in directory: /root/.cache/pip/wheels/76/48/9b/da67ff1a18fe8e9d428f9b1a177716d4a7d363d2bbe83bf6cf
Successfully built mysql.connector


In [ ]:
import mysql.connector
import json

# Conectar a la base de datos
conn = mysql.connector.connect(
    host="mysql-opset.alwaysdata.net",
    user="opset_us",
    password="Holamundo",
    database="opset_us"
)

# Variable de consulta a la base de datos
cursor = conn.cursor()

# Cargar datos desde el archivo JSON
with open('diccionario_maya.json','r',encoding='utf-8') as file:
  diccionario_data = json.load(file)

for palabra in diccionario_data:
  maya = palabra['Maya']
  espanol = palabra['Español']
  insert_query = "INSERT INTO diccionario_json (maya,espanol) VALUES(%s,%s)"
  cursor.execute(insert_query, (maya,espanol))

conn.commit()
cursor.close()
conn.close()


KeyboardInterrupt: 